# 재구축 데이터셋 Scratch



In [ ]:
import torch
import torch.utils.data as data
import torch.nn as nn
import torch.nn.functional as F
from torchvision import transforms, datasets, models
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data import TensorDataset
from PIL import Image
import numpy as np
from tqdm import tqdm

In [ ]:
# 구축된 .npy파일을 Pytorch DataLoader을 사용할 수 있도록 CUSTOM DATASET을 만듬.
import numpy as np
from google.colab import drive

drive.mount('/content/drive')
class CUB200(data.Dataset):
    def __init__(self, Train, transform = None):
        super(CUB200, self).__init__()
        """
        Train : bool = True
        """
        if Train == True:
            load_data = np.load('/content/drive/My Drive/colab_data/cub/final/train_data.npy')
            load_label = np.load('/content/drive/My Drive/colab_data/cub/final/train_label.npy')
            self.image = load_data
            self.label = load_label
            self.transform = transform
            
        else:
            load_data = np.load('/content/drive/My Drive/colab_data/cub/final/test_data.npy')
            #train data를 똑같이 쓰는거
            #or 트레인 데이터 반으로 나눠서 쓰기
            load_label = np.load('/content/drive/My Drive/colab_data/cub/final/test_label.npy')
            self.image = load_data
            self.label = load_label
            self.transform = transform

    def __getitem__(self, index):
        img, target = self.image[index], self.label[index]
        img = Image.fromarray(img)

        if self.transform is not None:
            img = self.transform(img)

        return img, target

    def __len__(self):
        return len(self.image)

Mounted at /content/drive


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [ ]:
# train_data에만 data augmentaion을 적용
transform_train = transforms.Compose([
        #이거 주석 풀면 데이터 어규먼테이션 가능
        # transforms.RandomCrop(224),
        transforms.ToTensor(),
        transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))])

transform_test = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))])

In [ ]:
# CUSTOM DATASET을 이용하여 train_loader, test_loader을 구축

batch_size = 8

train_loader = torch.utils.data.DataLoader(
    dataset = CUB200(True, transform = transform_train),
    batch_size = batch_size,
    shuffle = True
)

test_loader = torch.utils.data.DataLoader(
    dataset = CUB200(False, transform = transform_test),
    batch_size = batch_size,
    shuffle = False
)

In [ ]:
def training_model(model, criterion, optimizer, scheduler, num_epochs = 25):


    for epoch in range(num_epochs):
        scheduler.step()

        running_loss = 0.0
        for i, data in enumerate(train_loader, 0):
            inputs, labels = data
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()

            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

            if i % 20 == 19:
                print('[%d, %5d] loss: %.7f' %
                    (epoch + 1, (i + 1), running_loss / 20))
                running_loss = 0.0
        
        train_correct = 0
        train_total = 0
        for i, data in enumerate(train_loader, 0):
            inputs, labels = data
            inputs = inputs.squeeze()
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)

            _, predicted = torch.max(outputs.data, 1)
            train_total += labels.size(0)
            train_correct += (predicted == labels).sum().item()

        print('[%d epoch] Accuracy of the network on the train images: %d %%' %
              (epoch + 1, 100 * train_correct / train_total))
        
    print("End Training do it eval_accuracy")
    return model

In [ ]:
def eval_accuracy(model):
    #프레딕션 할떄 트레인 모델로 만든 딥러닝 모델이 프레딕션한 레이블을 리스트로 만들어서 암호화해 올리면 됨
    class_correct = list(0. for i in range(50))
    class_total = list(0. for i in range(50))

    correct = 0
    total = 0
    
    model.eval()
    with torch.no_grad():
        for i, data in enumerate(test_loader, 0):
            images, labels = data
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            c = (predicted == labels).squeeze()
                    
            for i in range(labels.shape[0]):
                label = labels[i]
                class_correct[label] += c[i].item()
                class_total[label] += 1
                
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

    print('Accuracy of the network on test images: %d %%' % (
        100 * correct / total))            
                
    return 

In [ ]:
class Scratch_Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.pool = nn.MaxPool2d(2, 2)
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=128, kernel_size=5, stride=1)
        self.conv2 = nn.Conv2d(in_channels=128, out_channels=256, kernel_size=5, stride=1)
        self.conv3 = nn.Conv2d(in_channels=256, out_channels=256, kernel_size=5, stride=1)
        self.conv4 = nn.Conv2d(in_channels=256, out_channels=128, kernel_size=3, stride=1)
        # self.conv5 = nn.Conv2d(in_channels=256, out_channels=128, kernel_size=3, stride=1)
        self.fc1 = nn.Linear(128 * 13 * 13, 512)
        self.fc2 = nn.Linear(512, 50)
        # self.fc3 = nn.Linear(256, 50)

    def forward(self, x):
        # print("연산 전", x.size()) #[8, 3, 224, 224]
        x = self.pool(F.relu(self.conv1(x)))
        # print("conv1 연산 후", x.size()) #[8, 128, 110, 110]
        x = self.pool(F.relu(self.conv2(x)))
        # print("conv2 연산 후",x.size()) # [8, 256, 55, 55]
        x = self.pool(F.relu(self.conv3(x)))
        # print("conv3 연산 후",x.size()) # [8, 256, 26, 26]
        x = self.pool(F.relu(self.conv4(x)))
        # print("conv4 연산 후",x.size()) # [8, 256, 26, 26]
        # x = self.pool(F.relu(self.conv5(x)))
        # print("conv5 연산 후",x.size()) # [8, 256, 26, 26]
        x = x.view(-1, 128 * 13 * 13)
        # print("차원 감소 후", x.size()) # [8, 18432, 1, 1]
        x = F.relu(self.fc1(x))
        # print("fc1 연산 후", x.size()) # [8, 512, 1, 1]
        # x = F.relu(self.fc2(x))
        # print("fc2 연산 후", x.size())
        x = self.fc2(x)
        # print("fc3 연산 후", x.size()) # [8, 50, 1, 1]
        return x


model_ft = Scratch_Net()
output = model_ft(torch.randn(8, 3, 256, 256))

In [ ]:
num_epochs = 25
model_ft.to(device)
criterion = nn.CrossEntropyLoss()
# optimizer = optim.Adam(model_ft.parameters(), lr = 0.001)
optimizer = optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9)
lr_scheduler = optim.lr_scheduler.StepLR(optimizer, step_size = 7, gamma = 0.1)

In [ ]:
model_ft = training_model(model_ft, criterion, optimizer, lr_scheduler, num_epochs)

/usr/local/lib/python3.7/dist-packages/torch/optim/lr_scheduler.py:134: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


[1,    20] loss: 3.9171464
[1,    40] loss: 3.9141876
[1,    60] loss: 3.9140873
[1,    80] loss: 3.9142504
[1,   100] loss: 3.9101089
[1 epoch] Accuracy of the network on the train images: 3 %
[2,    20] loss: 3.8982859
[2,    40] loss: 3.8910889
[2,    60] loss: 3.8954141
[2,    80] loss: 3.9026462
[2,   100] loss: 3.9048022
[2 epoch] Accuracy of the network on the train images: 7 %
[3,    20] loss: 3.8426473
[3,    40] loss: 3.8235675
[3,    60] loss: 3.8123295
[3,    80] loss: 3.8093471
[3,   100] loss: 3.8514423
[3 epoch] Accuracy of the network on the train images: 8 %
[4,    20] loss: 3.6842944
[4,    40] loss: 3.6595941
[4,    60] loss: 3.7709548
[4,    80] loss: 3.6936137
[4,   100] loss: 3.7190937
[4 epoch] Accuracy of the network on the train images: 9 %
[5,    20] loss: 3.5292681
[5,    40] loss: 3.4710718
[5,    60] loss: 3.4932063
[5,    80] loss: 3.5737743
[5,   100] loss: 3.4992071
[5 epoch] Accuracy of the network on the train images: 15 %
[6,    20] loss: 3.2156302
[6

In [ ]:
eval_accuracy(model_ft)

Accuracy of the network on the 500 test images: 13 %


'End model test'